# 2.2 - Web Scraping (bs4)

$$$$

![scraping](images/scraping.png)

$$$$

Web scraping o raspado web, es una técnica utilizada mediante programas de software para extraer información de sitios web. Usualmente, estos programas simulan la navegación de un humano en la web ya sea utilizando el protocolo HTTP manualmente, o incrustando un navegador en una aplicación.

El web scraping está muy relacionado con la indexación de la web, la cual indexa la información de la web utilizando un robot y es una técnica universal adoptada por la mayoría de los motores de búsqueda. Sin embargo, el web scraping se enfoca más en la transformación de datos sin estructura en la web, como el formato HTML, en datos estructurados que pueden ser almacenados y analizados en una base de datos central, en una hoja de cálculo o en alguna otra fuente de almacenamiento. Alguno de los usos del web scraping son la comparación de precios en tiendas, la monitorización de datos relacionados con el clima de cierta región, la detección de cambios en sitios webs y la integración de datos en sitios webs. 

En los últimos años el web scraping se ha convertido en una técnica muy utilizada dentro del sector del posicionamiento web gracias a su capacidad de generar grandes cantidades de datos para crear contenidos de calidad.

Podríamos pensar que el web scraping es nuestro último recurso a falta de una API o un feed RSS. A falta de una fuente de datos, siempre podemos extraer aquello que sale por pantalla.

### Extracción desde el HTML

Para scrapear una página web, en primer lugar debemos conocer las estructura que tiene el HTML. Veamos la estructura básica.

El HTML consiste en contenido `<etiquetado>`, es como si fueran cajas de contenido, organizado de manera jerárquica:

```
<html>
    <head>
        <title>Titulo de la pagina</title>
    </head>
    <body>
        <h1>Cabecera</h1>
        <p>Parrafo</p>
    </body>
</html>
```

$$$$

Las etiquetas el HTML se pueden clasificar en varios grupos, dependiendo del tipo de contenido que posea. Estos son algunos ejemplos:

+ cabecera: `<h1>`, `<h2>`, `<h3>`, `<hgroup>`...
+ texto: `<b>`, `<p>`...
+ embebido: `<audio>`, `<img>`, `<video>`...
+ tabular: `<table>`, `<tr>`, `<td>`, `<tbody>`...
+ secciones: `<header>`, `<section>`, `<article>`...
+ metadata: `<meta>`, `<title>`, `<script>`...

$$$$

Las etiquetas pueden tener atributos. Por ejemplo:
 
`<div class="text-monospace" id="name_132", href="www.example.com"> Contenido de la pagina </div>` 

Esta etiqueta `div` tiene los siguientes atributos:

+ class: atributo con valor "text-monospace". La clase no es única en la página.
+ id: atributo con valor "name_132". El id de una etiqueta la identifica de manera unívoca.
+ href: atributo con valor "www.example.com". El href suele contener el link a otra parte de la página.

Siguiendo con la analogía de las cajas, si una etiqueta de HTML es una caja, los atributos serían las pegatinas pegadas en la tapa de la caja.

Conociendo cual es el contenido que queremos extraer, debemos encontrar las etiquetas dentro del HTML de la página web.

Usaremos la herramienta **[BeautifulSoup](https://www.crummy.com/software/BeautifulSoup/bs4/doc/)**.

In [1]:
%pip install beautifulsoup4

Note: you may need to restart the kernel to use updated packages.


In [2]:
import requests as req

from bs4 import BeautifulSoup as bs   # ambos alias son cosa mia

### Ejemplos Wikipedia

**[Países europeos según esperanza de vida](https://en.wikipedia.org/wiki/List_of_European_countries_by_life_expectancy)**

In [3]:
url='https://en.wikipedia.org/wiki/List_of_European_countries_by_life_expectancy'

In [4]:
# usamos requests para extraer el html en string

html=req.get(url).content

html[:1000]

b'<!DOCTYPE html>\n<html class="client-nojs" lang="en" dir="ltr">\n<head>\n<meta charset="UTF-8"/>\n<title>List of European countries by life expectancy - Wikipedia</title>\n<script>document.documentElement.className="client-js";RLCONF={"wgBreakFrames":false,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"db82b803-6441-436c-877e-c52988a94f53","wgCSPNonce":false,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":false,"wgNamespaceNumber":0,"wgPageName":"List_of_European_countries_by_life_expectancy","wgTitle":"List of European countries by life expectancy","wgCurRevisionId":1092974666,"wgRevisionId":1092974666,"wgArticleId":22175559,"wgIsArticle":true,"wgIsRedirect":false,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Use dmy dates from April 2022","Articles with sho

In [5]:
soup=bs(html, 'html.parser')

type(soup)

bs4.BeautifulSoup

In [10]:
tabla=soup.find_all('table')[1]  # find_all(tag)

type(tabla)

bs4.element.Tag

In [19]:
filas=tabla.find_all('tr')

filas=[f.text.strip().split('\n') for f in filas]

filas[:5]

[['Countries',
  '',
  'all',
  '',
  'male',
  '',
  'female',
  '',
  'gendergap',
  '',
  'Δ2014',
  '',
  'Δ2000'],
 [''],
 ['Liechtenstein', '84.16', '82.60', '85.80', '3.20', '2.09', '7.33'],
 ['Switzerland', '83.90', '82.10', '85.80', '3.70', '0.71', '4.22'],
 ['Spain', '83.83', '81.10', '86.70', '5.60', '0.60', '4.87']]

In [21]:
# minima limpieza

final=[]

for f in filas:
    
    tmp=[]
    
    for palabra in f:
        
        if palabra!='':
            
            tmp.append(palabra)
            
    final.append(tmp)
    
final[:5]

[['Countries', 'all', 'male', 'female', 'gendergap', 'Δ2014', 'Δ2000'],
 [],
 ['Liechtenstein', '84.16', '82.60', '85.80', '3.20', '2.09', '7.33'],
 ['Switzerland', '83.90', '82.10', '85.80', '3.70', '0.71', '4.22'],
 ['Spain', '83.83', '81.10', '86.70', '5.60', '0.60', '4.87']]

In [23]:
import pandas as pd

col_names=final[0]

data=final[2:]

df=pd.DataFrame(data, columns=col_names)

df.head(10)

,Countries,all,male,female,gendergap,Δ2014,Δ2000
0,Liechtenstein,84.16,82.60,85.80,3.20,2.09,7.33
1,Switzerland,83.90,82.10,85.80,3.70,0.71,4.22
2,Spain,83.83,81.10,86.70,5.60,0.60,4.87
3,Italy,83.50,81.40,85.70,4.30,0.41,3.72
4,Iceland,83.16,81.70,84.70,3.00,0.30,3.51
5,Sweden,83.11,81.50,84.80,3.30,0.86,3.47
6,Norway,82.96,81.30,84.70,3.40,0.86,4.32
7,Malta,82.86,81.20,84.60,3.40,0.81,4.51
8,France,82.83,79.90,85.90,6.00,0.11,3.77
9,Ireland,82.70,80.80,84.70,3.90,1.35,6.17


In [34]:
# lo mismo de otra manera, con bs4

filas=tabla.find_all('tr')

columnas=[e.text.strip() for e in filas[0].find_all('th')]

data_1=[[e.text.strip() for e in f.find_all('td')] for f in filas[2:]]

columnas

['Countries', 'all', 'male', 'female', 'gendergap', 'Δ2014', 'Δ2000']

In [35]:
df=pd.DataFrame(data_1, columns=columnas)

df.head(10)

,Countries,all,male,female,gendergap,Δ2014,Δ2000
0,Liechtenstein,84.16,82.60,85.80,3.20,2.09,7.33
1,Switzerland,83.90,82.10,85.80,3.70,0.71,4.22
2,Spain,83.83,81.10,86.70,5.60,0.60,4.87
3,Italy,83.50,81.40,85.70,4.30,0.41,3.72
4,Iceland,83.16,81.70,84.70,3.00,0.30,3.51
5,Sweden,83.11,81.50,84.80,3.30,0.86,3.47
6,Norway,82.96,81.30,84.70,3.40,0.86,4.32
7,Malta,82.86,81.20,84.60,3.40,0.81,4.51
8,France,82.83,79.90,85.90,6.00,0.11,3.77
9,Ireland,82.70,80.80,84.70,3.90,1.35,6.17


In [41]:
df_control_c=pd.read_clipboard()

df_control_c.head()

,Rank,Country,Life,expectancy[6],Influenza,vaccination,"rate,",people,aged,65,and,"over,",2016,(%)[7]
0,1,Monaco[8],89.4,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,San,Marino[9],83.4,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,Switzerland,83.0,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,Spain,82.8,56%,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,Liechtenstein,82.7,28%,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [43]:
df.to_clipboard()

### Ejemplo geolocalización por IP

https://tools.keycdn.com/geo

**¿Dónde estoy?**

In [44]:
url='https://tools.keycdn.com/geo?host=166.123.4.66'

In [45]:
# mismo proceso de antes

html=req.get(url).content   # contenido en string de la pagina

sopa=bs(html, 'html.parser')  # contenido en objeto bs4 de la pagina

In [48]:
sopa.find('div', id='geoResult')

<div class="mt-4" id="geoResult">
<div class="bg-light medium rounded p-3">
<p class="small text-uppercase text-muted font-weight-semi-bold line-height-headings mb-2">Location</p> <dl class="row mb-0">
<dt class="col-4">Country</dt><dd class="col-8 text-monospace">United States (US)</dd><dt class="col-4">Continent</dt><dd class="col-8 text-monospace">North America (NA)</dd><dt class="col-4">Coordinates</dt><dd class="col-8 text-monospace">37.751 (lat) / -97.822 (long)</dd><dt class="col-4">Time</dt><dd class="col-8 text-monospace">2022-06-21 04:56:30 (America/Chicago)</dd> </dl>
<p class="small text-uppercase text-muted font-weight-semi-bold line-height-headings mt-4 mb-2">Network</p>
<dl class="row mb-0">
<dt class="col-4">IP address</dt><dd class="col-8 text-monospace">166.123.4.66</dd><dt class="col-4">Hostname</dt><dd class="col-8 text-monospace">166.123.4.66</dd> </dl>
</div>
<div class="mapael mt-4" id="geoMap">
<div class="map position-relative"></div>
</div>
</div>

In [50]:
sopa.find('div', {'id': 'geoResult', 'class': 'mt-4'})

<div class="mt-4" id="geoResult">
<div class="bg-light medium rounded p-3">
<p class="small text-uppercase text-muted font-weight-semi-bold line-height-headings mb-2">Location</p> <dl class="row mb-0">
<dt class="col-4">Country</dt><dd class="col-8 text-monospace">United States (US)</dd><dt class="col-4">Continent</dt><dd class="col-8 text-monospace">North America (NA)</dd><dt class="col-4">Coordinates</dt><dd class="col-8 text-monospace">37.751 (lat) / -97.822 (long)</dd><dt class="col-4">Time</dt><dd class="col-8 text-monospace">2022-06-21 04:56:30 (America/Chicago)</dd> </dl>
<p class="small text-uppercase text-muted font-weight-semi-bold line-height-headings mt-4 mb-2">Network</p>
<dl class="row mb-0">
<dt class="col-4">IP address</dt><dd class="col-8 text-monospace">166.123.4.66</dd><dt class="col-4">Hostname</dt><dd class="col-8 text-monospace">166.123.4.66</dd> </dl>
</div>
<div class="mapael mt-4" id="geoMap">
<div class="map position-relative"></div>
</div>
</div>

In [54]:
sopa.find('div', class_='mt-4')

<div class="col-xl-2 col-md-3 d-flex align-items-end mt-md-0 mt-4">
<button class="btn btn-primary d-flex justify-content-center align-items-center w-md-100" id="geoBtn">Find</button>
</div>

In [56]:
tabla=sopa.find('div', {'id': 'geoResult'})

tabla.find_all('dd', {'class': 'col-8 text-monospace'})

[<dd class="col-8 text-monospace">United States (US)</dd>,
 <dd class="col-8 text-monospace">North America (NA)</dd>,
 <dd class="col-8 text-monospace">37.751 (lat) / -97.822 (long)</dd>,
 <dd class="col-8 text-monospace">2022-06-21 04:56:30 (America/Chicago)</dd>,
 <dd class="col-8 text-monospace">166.123.4.66</dd>,
 <dd class="col-8 text-monospace">166.123.4.66</dd>]

In [57]:
conexion=[e.text for e in tabla.find_all('dd', {'class': 'col-8 text-monospace'})]

conexion

['United States (US)',
 'North America (NA)',
 '37.751 (lat) / -97.822 (long)',
 '2022-06-21 04:56:30 (America/Chicago)',
 '166.123.4.66',
 '166.123.4.66']

In [59]:
tabla.find_all('dd', {'class': 'col-8 text-monospace'})[0].text  # text devuelve el contenido de la caja

'United States (US)'

In [60]:
# creo funcion para todo el proceso

def encontrar(ip):
    
    url=f'https://tools.keycdn.com/geo?host={ip}'
    
    html=req.get(url).content   # contenido en string de la pagina

    sopa=bs(html, 'html.parser')  # contenido en objeto bs4 de la pagina
    
    tabla=sopa.find('div', {'id': 'geoResult'})
    
    conexion=[e.text for e in tabla.find_all('dd', {'class': 'col-8 text-monospace'})]

    return conexion

In [62]:
encontrar('168.123.4.5')

['Guam (GU)',
 'Oceania (OC)',
 '13.4786 (lat) / 144.8183 (long)',
 '2022-06-21 20:03:17 (Pacific/Guam)',
 '168.123.4.5',
 '168.123.4.5',
 'UNIVERSITY-GUAM',
 '395400']

### Ejemplo LinkedIn

In [63]:
url='https://www.linkedin.com/jobs/search/?distance=25.0&f_WT=2&geoId=105646813&keywords=An%C3%A1lisis%20de%20datos'

In [64]:
html=req.get(url).content

sopa=bs(html, 'html.parser')

In [80]:
todas_tarjetas=sopa.find('ul', class_="jobs-search__results-list")

In [83]:
data_1_curro=todas_tarjetas.find_all('li')[0]

In [107]:
titulo=data_1_curro.find('h3').text.strip()

empresa=data_1_curro.find('a', class_="hidden-nested-link").text.strip()

pais=data_1_curro.find('span', class_="job-search-card__location").text.strip()

link_empresa=data_1_curro.find('a', class_="hidden-nested-link").attrs['href']

link_curro=data_1_curro.find('a').attrs['href']

fecha_publi=data_1_curro.find('time').attrs['datetime']

In [109]:
{'titulo': titulo,
 'empresa': empresa,
 'pais': pais,
 'link_empresa': link_empresa,
 'link_curro': link_curro,
 'fecha_publi': fecha_publi}

{'titulo': 'Machine Learning',
 'empresa': 'Prima Assicurazioni',
 'pais': 'Spain',
 'link_empresa': 'https://it.linkedin.com/company/prima-assicurazioni?trk=public_jobs_jserp-result_job-search-card-subtitle',
 'link_curro': 'https://es.linkedin.com/jobs/view/machine-learning-at-prima-assicurazioni-3037602820?refId=QebuNq69JfZltXfHJk7JeA%3D%3D&trackingId=wZFs68iZwxGLQGp3IxKg0A%3D%3D&position=1&pageNum=0&trk=public_jobs_jserp-result_search-card',
 'fecha_publi': '2022-06-01'}

In [117]:
def linkedin(keywords, loc, n_secs, exp, num_pages, j_type):
    
    URL='https://www.linkedin.com/jobs/search/'
    
    data=[]
    
    for i in range(num_pages):
        
        scrape_url=''.join([URL,  # url base
                            f'?keywords={keywords}',  # palabras clave
                            f'&location={loc}',       # pais
                            f'&f_WT={j_type}',         # remoto hibrido presencial
                            f'&f_TPR={n_secs}',       # tiempo atras
                            f'&F_E={exp}',            # experiencia
                            f'&start={i*25}'          # i=numero de paginas, 25 trabajos por pagina
                           ])
        print(scrape_url)
        
        html=req.get(url).content

        sopa=bs(html, 'html.parser')
        
        for oferta in sopa.find('ul', class_="jobs-search__results-list").find_all('li'):
            
            # bucle para todas las ofertas
            titulo=oferta.find('h3').text.strip()

            empresa=oferta.find('a', class_="hidden-nested-link").text.strip()

            pais=oferta.find('span', class_="job-search-card__location").text.strip()

            link_empresa=oferta.find('a', class_="hidden-nested-link").attrs['href']

            link_curro=oferta.find('a').attrs['href']

            fecha_publi=oferta.find('time').attrs['datetime']
            
            data.append({'titulo': titulo,
                         'empresa': empresa,
                         'pais': pais,
                         'location': loc, 
                         'exp': exp,
                         'j_type': j_type,
                         'link_empresa': link_empresa,
                         'link_curro': link_curro,
                         'fecha_publi': fecha_publi})
           
        
    return pd.DataFrame(data)

In [119]:
df=linkedin('data', 'spain', 1e6, 4, 4, 1)

https://www.linkedin.com/jobs/search/?keywords=data&location=spain&f_WT=1&f_TPR=1000000.0&F_E=4&start=0
https://www.linkedin.com/jobs/search/?keywords=data&location=spain&f_WT=1&f_TPR=1000000.0&F_E=4&start=25
https://www.linkedin.com/jobs/search/?keywords=data&location=spain&f_WT=1&f_TPR=1000000.0&F_E=4&start=50
https://www.linkedin.com/jobs/search/?keywords=data&location=spain&f_WT=1&f_TPR=1000000.0&F_E=4&start=75


In [120]:
df.head()

,titulo,empresa,pais,location,exp,j_type,link_empresa,link_curro,fecha_publi
0,Business Analyst,Randstad,"Madrid, Community of Madrid, Spain",spain,4,1,https://nl.linkedin.com/company/randstad?trk=p...,https://es.linkedin.com/jobs/view/business-ana...,2022-06-20
1,Product Analyst,Alooba,Spain,spain,4,1,https://au.linkedin.com/company/alooba?trk=pub...,https://es.linkedin.com/jobs/view/product-anal...,2022-06-20
2,Business Analyst,atSistemas,Spain,spain,4,1,https://es.linkedin.com/company/atsistemas?trk...,https://es.linkedin.com/jobs/view/business-ana...,2022-05-30
3,Product Analyst,IZERTIS,Spain,spain,4,1,https://es.linkedin.com/company/izertis?trk=pu...,https://es.linkedin.com/jobs/view/product-anal...,2022-06-17
4,Business Analyst,IC Resources,Spain,spain,4,1,https://uk.linkedin.com/company/icresources?tr...,https://es.linkedin.com/jobs/view/business-ana...,2022-05-09


In [123]:
from IPython.display import HTML

HTML(df.to_html(render_links=True))

,titulo,empresa,pais,location,exp,j_type,link_empresa,link_curro,fecha_publi
0,Business Analyst,Randstad,"Madrid, Community of Madrid, Spain",spain,4,1,https://nl.linkedin.com/company/randstad?trk=public_jobs_jserp-result_job-search-card-subtitle,https://es.linkedin.com/jobs/view/business-analyst-at-randstad-3132783358?refId=F7P5y028NVELxZAMMnUuXg%3D%3D&trackingId=35pK37Z%2BBSrG1sfiWYoSkw%3D%3D&position=1&pageNum=0&trk=public_jobs_jserp-result_search-card,2022-06-20
1,Product Analyst,Alooba,Spain,spain,4,1,https://au.linkedin.com/company/alooba?trk=public_jobs_jserp-result_job-search-card-subtitle,https://es.linkedin.com/jobs/view/product-analyst-at-alooba-3126998043?refId=F7P5y028NVELxZAMMnUuXg%3D%3D&trackingId=Z5XTbEOsvgxphnmjBBouxw%3D%3D&position=2&pageNum=0&trk=public_jobs_jserp-result_search-card,2022-06-20
2,Business Analyst,atSistemas,Spain,spain,4,1,https://es.linkedin.com/company/atsistemas?trk=public_jobs_jserp-result_job-search-card-subtitle,https://es.linkedin.com/jobs/view/business-analyst-at-atsistemas-3099490731?refId=F7P5y028NVELxZAMMnUuXg%3D%3D&trackingId=cIncOE%2Bz8Erd9AzEzzyyRw%3D%3D&position=3&pageNum=0&trk=public_jobs_jserp-result_search-card,2022-05-30
3,Product Analyst,IZERTIS,Spain,spain,4,1,https://es.linkedin.com/company/izertis?trk=public_jobs_jserp-result_job-search-card-subtitle,https://es.linkedin.com/jobs/view/product-analyst-at-izertis-3129398847?refId=F7P5y028NVELxZAMMnUuXg%3D%3D&trackingId=w5TAVVvzaoG4TgTkR91bqA%3D%3D&position=4&pageNum=0&trk=public_jobs_jserp-result_search-card,2022-06-17
4,Business Analyst,IC Resources,Spain,spain,4,1,https://uk.linkedin.com/company/icresources?trk=public_jobs_jserp-result_job-search-card-subtitle,https://es.linkedin.com/jobs/view/business-analyst-at-ic-resources-2945933065?refId=F7P5y028NVELxZAMMnUuXg%3D%3D&trackingId=3EGhctVK9a%2FTdGlrf%2BWmKg%3D%3D&position=5&pageNum=0&trk=public_jobs_jserp-result_search-card,2022-05-09
5,Data Risk Manager,Hidden Talent,Spain,spain,4,1,https://es.linkedin.com/company/hiddent?trk=public_jobs_jserp-result_job-search-card-subtitle,https://es.linkedin.com/jobs/view/data-risk-manager-at-hidden-talent-3133158842?refId=F7P5y028NVELxZAMMnUuXg%3D%3D&trackingId=g2T5DlFfvtAuTwea%2BV3uSQ%3D%3D&position=6&pageNum=0&trk=public_jobs_jserp-result_search-card,2022-06-20
6,Business Analyst,Luxoft,Spain,spain,4,1,https://ch.linkedin.com/company/luxoft?trk=public_jobs_jserp-result_job-search-card-subtitle,https://es.linkedin.com/jobs/view/business-analyst-at-luxoft-3116783222?refId=F7P5y028NVELxZAMMnUuXg%3D%3D&trackingId=xEbLyqD4X4t%2FbrhWnj8C0A%3D%3D&position=7&pageNum=0&trk=public_jobs_jserp-result_search-card,2022-06-08
7,Business Analyst,Unisys,"Madrid, Community of Madrid, Spain",spain,4,1,https://www.linkedin.com/company/unisys?trk=public_jobs_jserp-result_job-search-card-subtitle,https://es.linkedin.com/jobs/view/business-analyst-at-unisys-3101748633?refId=F7P5y028NVELxZAMMnUuXg%3D%3D&trackingId=uJlFlVLn0JyUMtPptNJ6Cg%3D%3D&position=8&pageNum=0&trk=public_jobs_jserp-result_search-card,2022-05-31
8,Business Analyst,Umanis,Spain,spain,4,1,https://fr.linkedin.com/company/umanis?trk=public_jobs_jserp-result_job-search-card-subtitle,https://es.linkedin.com/jobs/view/business-analyst-at-umanis-3116288140?refId=F7P5y028NVELxZAMMnUuXg%3D%3D&trackingId=PYpWkMnSo3lgeC1xGx%2FXYA%3D%3D&position=9&pageNum=0&trk=public_jobs_jserp-result_search-card,2022-06-09
9,Business Analyst,Volt - International,"Barcelona, Catalonia, Spain",spain,4,1,https://uk.linkedin.com/company/volt-international?trk=public_jobs_jserp-result_job-search-card-subtitle,https://es.linkedin.com/jobs/view/business-analyst-at-volt-international-3129477057?refId=F7P5y028NVELxZAMMnUuXg%3D%3D&trackingId=J%2BXCH1OBIrMX6D2p2INNGA%3D%3D&position=10&pageNum=0&trk=public_jobs_jserp-result_search-card,2022-06-17
